# Example of using simsopt to optimize simsgeo objects

This notebook demonstrates a silly little example of optimizing a simsgeo curve using simsopt. We start with a curve that has random Fourier coefficients for $R(\phi)$ and $Z(\phi)$, where $(R, \phi, Z)$ are cylindrical coordinates. We minimize the length of the curve, keeping the 0-frequency Fourier mode of $R$ fixed. The result should be a circle.

As of now, one change is needed in simsgeo to get this to work: in `simsgeo/simsgeo/magneticaxis.py`, `set_dofs()` should call `self.invalidate_cache()`.

In [1]:
import numpy as np
from simsopt.geo.curverzfourier import CurveRZFourier
from simsopt.geo.objectives import CurveLength
from simsopt.core.least_squares_problem import LeastSquaresProblem
from simsopt.solve.serial_solve import least_squares_serial_solve


Initialize the simsgeo curve:

In [2]:
nquadrature = 100
nfourier = 4
nfp = 5
curve = CurveRZFourier(nquadrature, nfourier, nfp, True)

Initialize the Fourier coefficients to random numbers. The first coefficient is the average major radius, so set it to a positive number.

In [3]:
x0 = np.random.rand(curve.num_dofs()) - 0.5
x0[0] = 3.0
curve.set_dofs(x0)
print('Initial curve dofs: ', curve.get_dofs())

Initial curve dofs:  [ 3.          0.1062567  -0.39132479 -0.32167043 -0.19296166  0.42334101
  0.46584917 -0.21130773  0.14268142]


Tell the curve object that the first Fourier mode is fixed, whereas all the other dofs are not.

In [4]:
curve.all_fixed(False)
curve.fixed[0] = True

Presently in simsgeo, the length objective is a separate object rather than as a function of Curve itself, so we must create this 2nd object:

In [5]:
obj = optimizable(CurveLength(curve))
print('Initial curve length: ', obj.J())

Initial curve length:  45.79711879843295


The following assignment is needed for simsopt to recognize that `obj` depends on `curve`. Hopefully this step can be made unnecessary with a change in simsgeo.

In [6]:
obj.depends_on = ['curve']

Create a term in the least-squares objective function, $1.0 * (length - 0.0)^2$:

In [7]:
term1 = (obj, 0.0, 1.0)

Put this term into a least-squares problem object:

In [8]:
prob = LeastSquaresProblem([term1])

Simsopt recognizes that derivative information is available:

In [9]:
prob.dofs.grad_avail

True

Let's view the initial global state vector. It should contain all the initial Fourier modes except the first:

In [10]:
prob.x

array([ 0.1062567 , -0.39132479, -0.32167043, -0.19296166,  0.42334101,
        0.46584917, -0.21130773,  0.14268142])

Solve the minimization problem:

In [11]:
least_squares_serial_solve(prob)

Using derivatives
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.0487e+03                                    1.31e+03    
       1              2         7.7409e+02      2.75e+02       8.75e-01       2.06e+03    
       2              4         3.4483e+02      4.29e+02       2.19e-01       7.91e+02    
       3              6         2.4911e+02      9.57e+01       1.09e-01       2.77e+02    
       4              7         2.3543e+02      1.37e+01       2.19e-01       5.27e+02    
       5              8         2.0501e+02      3.04e+01       5.47e-02       2.27e+02    
       6              9         2.0149e+02      3.53e+00       1.09e-01       2.99e+02    
       7             10         1.9271e+02      8.78e+00       2.74e-02       1.45e+02    
       8             11         1.8902e+02      3.69e+00       5.47e-02       1.27e+02    
       9             12         1.8732e+02      1.70e+00       5.47e-02 

Examine properties of the optimum found:

In [12]:
print('At the optimum, x: ', prob.x)
print(' Final curve dofs: ', curve.get_dofs())
print(' Final curve length:    ', obj.J())
print(' Expected final length: ', 2 * np.pi * x0[0])
print(' objective function: ', prob.objective())

At the optimum, x:  [ 2.12639823e-05  8.06953109e-10  1.43494849e-11  7.16303994e-06
  1.38570670e-04  4.62734220e-10 -5.04666247e-11 -4.49856097e-06]
 Final curve dofs:  [ 3.00000000e+00  2.12639823e-05  8.06953109e-10  1.43494849e-11
  7.16303994e-06  1.38570670e-04  4.62734220e-10 -5.04666247e-11
 -4.49856097e-06]
 Final curve length:     18.849556193793447
 Expected final length:  18.84955592153876
 objective function:  355.3057687029769


All the Fourier modes after the first (the 0-frequency mode) have been reduced towards 0. The final curve length is approximately that of the circle with the expected radius.